# Composite and Iterator Pattern

* Overview:
* Iterator Pattern: This pattern provides a way to access elements of a collection sequentially without exposing the underlying representation.
* Composite Pattern: This pattern allows you to treat individual objects and compositions of objects uniformly, making it easier to work with tree structures.

We'll combine these patterns in a robust example to demonstrate how they can be used together.

## Example 01: File System Component

In [1]:
from abc import ABC, abstractmethod
from collections.abc import Iterable, Iterator

# Component (Abstract Class)
class FileSystemComponent(ABC):
    @abstractmethod
    def get_name(self):
        pass

    @abstractmethod
    def display(self, indent=0):
        pass


# Leaf (File)
class File(FileSystemComponent):
    def __init__(self, name: str):
        self.name = name

    def get_name(self):
        return self.name

    def display(self, indent=0):
        print(' ' * indent + self.get_name())


# Composite (Directory)
class Directory(FileSystemComponent):
    def __init__(self, name: str):
        self.name = name
        self.children = []

    def add(self, component: FileSystemComponent):
        self.children.append(component)

    def remove(self, component: FileSystemComponent):
        self.children.remove(component)

    def get_name(self):
        return self.name

    def display(self, indent=0):
        print(' ' * indent + self.get_name())
        for child in self.children:
            child.display(indent + 2)

    def __iter__(self):
        return CompositeIterator(self.children)


# Iterator Pattern Implementation

class CompositeIterator(Iterator):
    def __init__(self, children):
        self.stack = [iter(children)]

    def __next__(self):
        if not self.stack:
            raise StopIteration

        iterator = self.stack[-1]

        try:
            component = next(iterator)

            if isinstance(component, Directory):
                self.stack.append(iter(component.children))

            return component
        except StopIteration:
            self.stack.pop()
            return next(self)

    def __iter__(self):
        return self


# Client Code
if __name__ == "__main__":
    # Create Files
    file1 = File("File1.txt")
    file2 = File("File2.txt")
    file3 = File("File3.txt")

    # Create Directories and add files/directories
    root_dir = Directory("root")
    home_dir = Directory("home")
    user_dir = Directory("user")

    root_dir.add(home_dir)
    home_dir.add(user_dir)
    user_dir.add(file1)
    user_dir.add(file2)
    root_dir.add(file3)

    # Display the structure
    print("File System Structure:")
    root_dir.display()

    # Traverse the structure using the Iterator
    print("\nIterating over the File System:")
    for component in root_dir:
        print(f"Visited: {component.get_name()}")


File System Structure:
root
  home
    user
      File1.txt
      File2.txt
  File3.txt

Iterating over the File System:
Visited: home
Visited: user
Visited: File1.txt
Visited: File2.txt
Visited: File3.txt


## Example 02: Menu Item and Pacake House

In [3]:
class MenuItem:
    def __init__(self, name: str, description: str, vegetarian: bool, price: float):
        self.name = name
        self.description = description
        self.vegetarian = vegetarian
        self.price = price

    def get_name(self):
        return self.name

    def get_description(self):
        return self.description

    def is_vegetarian(self):
        return self.vegetarian

    def get_price(self):
        return self.price

    def __str__(self):
        return f"{self.name}, {self.price} -- {self.description}"


from collections.abc import Iterator, Iterable

class PancakeHouseMenu(Iterable):
    def __init__(self):
        self.menu_items = []
        self.add_item("K&B's Pancake Breakfast", "Pancakes with scrambled eggs, and toast", True, 2.99)
        self.add_item("Regular Pancake Breakfast", "Pancakes with fried eggs, sausage", False, 2.99)
        self.add_item("Blueberry Pancakes", "Pancakes made with fresh blueberries", True, 3.49)
        self.add_item("Waffles", "Waffles, with your choice of blueberries or strawberries", True, 3.59)

    def add_item(self, name: str, description: str, vegetarian: bool, price: float):
        menu_item = MenuItem(name, description, vegetarian, price)
        self.menu_items.append(menu_item)

    def __iter__(self):
        return iter(self.menu_items)

    def __str__(self):
        return "\n".join(str(item) for item in self.menu_items)

class PancakeHouseMenuIterator(Iterator):
    def __init__(self, items):
        self.items = items
        self.position = 0

    def __next__(self):
        if self.position >= len(self.items):
            raise StopIteration
        menu_item = self.items[self.position]
        self.position += 1
        return menu_item

    def __iter__(self):
        return self


if __name__ == "__main__":
    pancake_menu = PancakeHouseMenu()
    
    # Display all menu items
    print("Menu:")
    print(pancake_menu)

    # Iterating over menu items using the built-in iterator
    print("\nIterating over menu items:")
    for item in pancake_menu:
        print(item)

    # Alternatively, using a custom iterator
    print("\nUsing custom iterator:")
    custom_iterator = PancakeHouseMenuIterator(pancake_menu.menu_items)
    for item in custom_iterator:
        print(item)

Menu:
K&B's Pancake Breakfast, 2.99 -- Pancakes with scrambled eggs, and toast
Regular Pancake Breakfast, 2.99 -- Pancakes with fried eggs, sausage
Blueberry Pancakes, 3.49 -- Pancakes made with fresh blueberries
Waffles, 3.59 -- Waffles, with your choice of blueberries or strawberries

Iterating over menu items:
K&B's Pancake Breakfast, 2.99 -- Pancakes with scrambled eggs, and toast
Regular Pancake Breakfast, 2.99 -- Pancakes with fried eggs, sausage
Blueberry Pancakes, 3.49 -- Pancakes made with fresh blueberries
Waffles, 3.59 -- Waffles, with your choice of blueberries or strawberries

Using custom iterator:
K&B's Pancake Breakfast, 2.99 -- Pancakes with scrambled eggs, and toast
Regular Pancake Breakfast, 2.99 -- Pancakes with fried eggs, sausage
Blueberry Pancakes, 3.49 -- Pancakes made with fresh blueberries
Waffles, 3.59 -- Waffles, with your choice of blueberries or strawberries
